In [1]:
import pandas as pd
import numpy as np
import sqlite3 as lite

# 1. Groupby 

- SQL groupby 명령어와 같음  
- split -> apply -> combine 과정을 거쳐 연산함  

In [2]:
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}

df = pd.DataFrame(ipl_data)
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [3]:
df.groupby('Team')['Points'].sum() # Team끼리 Groupby하고 Point별 sum을 구해라

Team
Devils    1536
Kings     2285
Riders    3049
Royals    1505
kings      812
Name: Points, dtype: int64

- 한 개 이상의 column을 묶을 수 있음

In [4]:
df.groupby(['Team', 'Year'])['Points'].sum()

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
kings   2015    812
Name: Points, dtype: int64

## 1-1) Hierarchical Index
- groupby 명령의 결과물도 결국은 dataframe  
- 두 개의 column으로 groupby를 할 경우, index가 두개 생성

In [6]:
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [7]:
h_index = df.groupby(["Team", "Year"])["Points"].sum()
h_index

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
kings   2015    812
Name: Points, dtype: int64

In [8]:
h_index.index

MultiIndex(levels=[['Devils', 'Kings', 'Riders', 'Royals', 'kings'], [2014, 2015, 2016, 2017]],
           labels=[[0, 0, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4], [0, 1, 0, 2, 3, 0, 1, 2, 3, 0, 1, 1]],
           names=['Team', 'Year'])

In [9]:
h_index["Devils":"Kings"]

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Name: Points, dtype: int64

### 1-1-1) unstack()
- Group으로 묶여진 데이터를 matrix형태로 전환해줌

In [10]:
h_index.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,NaN,NaN
Kings,741.0,NaN,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,NaN,NaN
kings,NaN,812.0,NaN,NaN


In [16]:
h_index.unstack().fillna(0) # NaN값에 0으로 채움

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,0.0,0.0
Kings,741.0,0.0,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,0.0,0.0
kings,0.0,812.0,0.0,0.0


### 1-1-2) swaplevel
- Index level을 변경할 수 있음

In [11]:
h_index.swaplevel()

Year  Team  
2014  Devils    863
2015  Devils    673
2014  Kings     741
2016  Kings     756
2017  Kings     788
2014  Riders    876
2015  Riders    789
2016  Riders    694
2017  Riders    690
2014  Royals    701
2015  Royals    804
      kings     812
Name: Points, dtype: int64

In [13]:
h_index.swaplevel().sortlevel(0) # Year별로 sorting 해줌

c:\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: sortlevel is deprecated, use sort_index(level=...)
  """Entry point for launching an IPython kernel.


Year  Team  
2014  Devils    863
      Kings     741
      Riders    876
      Royals    701
2015  Devils    673
      Riders    789
      Royals    804
      kings     812
2016  Kings     756
      Riders    694
2017  Kings     788
      Riders    690
Name: Points, dtype: int64

### 1-1-3) Operations
- Index level을 기준으로 기본 연산 수행 가능

In [14]:
h_index.sum(level=0)

Team
Devils    1536
Kings     2285
Riders    3049
Royals    1505
kings      812
Name: Points, dtype: int64

In [15]:
h_index.sum(level=1)

Year
2014    3181
2015    3078
2016    1450
2017    1478
Name: Points, dtype: int64

## 1-2) Grouped
- Groupby에 의해 split된 상태를 추출 가능함

In [17]:
grouped = df.groupby('Team')

for name, group in grouped:
    print(name)
    print(group)
    
# Tuple형태로 그룹의 key, value 값 추출

Devils
     Team  Rank  Year  Points
2  Devils     2  2014     863
3  Devils     3  2015     673
Kings
    Team  Rank  Year  Points
4  Kings     3  2014     741
6  Kings     1  2016     756
7  Kings     1  2017     788
Riders
      Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
8   Riders     2  2016     694
11  Riders     2  2017     690
Royals
      Team  Rank  Year  Points
9   Royals     4  2014     701
10  Royals     1  2015     804
kings
    Team  Rank  Year  Points
5  kings     4  2015     812


In [18]:
for data in grouped:
    print(data)

('Devils',      Team  Rank  Year  Points
2  Devils     2  2014     863
3  Devils     3  2015     673)
('Kings',     Team  Rank  Year  Points
4  Kings     3  2014     741
6  Kings     1  2016     756
7  Kings     1  2017     788)
('Riders',       Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
8   Riders     2  2016     694
11  Riders     2  2017     690)
('Royals',       Team  Rank  Year  Points
9   Royals     4  2014     701
10  Royals     1  2015     804)
('kings',     Team  Rank  Year  Points
5  kings     4  2015     812)


- 특정 key값을 가진 그룹의 정보만 추출 가능

In [19]:
grouped.get_group('Devils')

,Team,Rank,Year,Points
2,Devils,2,2014,863
3,Devils,3,2015,673


- 추출된 group 정보에는 세 가지 유형의 apply가 가능함  
- Aggregation : 요약된 통계정보를 추출해줌  
- Transformation : 해당 정보를 변환해줌  
- Filtration : 특정 정보를 제거하여 보여주는 필터링 기능

### 1-2-1) agg : aggregation

In [20]:
grouped.agg(sum)

,Rank,Year,Points
Team,,,
Devils,5,4029,1536
Kings,5,6047,2285
Riders,7,8062,3049
Royals,5,4029,1505
kings,4,2015,812


In [22]:
# import numpy as np
grouped.agg(np.mean)

,Rank,Year,Points
Team,,,
Devils,2.500000,2014.500000,768.000000
Kings,1.666667,2015.666667,761.666667
Riders,1.750000,2015.500000,762.250000
Royals,2.500000,2014.500000,752.500000
kings,4.000000,2015.000000,812.000000


In [23]:
grouped.agg(min)

,Rank,Year,Points
Team,,,
Devils,2,2014,673
Kings,1,2014,741
Riders,1,2014,690
Royals,1,2014,701
kings,4,2015,812


In [24]:
grouped.agg(max)

,Rank,Year,Points
Team,,,
Devils,3,2015,863
Kings,3,2017,788
Riders,2,2017,876
Royals,4,2015,804
kings,4,2015,812


- 특정 column에 여러개의 function을 apply할 수도 있음

In [29]:
grouped['Points'].agg([np.sum, np.mean, np.std]).fillna(0)

,sum,mean,std
Team,,,
Devils,1536,768.000000,134.350288
Kings,2285,761.666667,24.006943
Riders,3049,762.250000,88.567771
Royals,1505,752.500000,72.831998
kings,812,812.000000,0.000000


### 1-2-2) transformation
- Aggregation과 달리 key값 별로 요약된 정보가 아님  
- 개별 데이터의 변환을 지원함

In [30]:
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


- 바로는 크게 쓰이지 않음.  

In [31]:
score = lambda x: (x)
grouped.transform(score)

,Rank,Year,Points
0,1,2014,876
1,2,2015,789
2,2,2014,863
3,3,2015,673
4,3,2014,741
5,4,2015,812
6,1,2016,756
7,1,2017,788
8,2,2016,694
9,4,2014,701


- 단 max나 min처럼 Series 데이터에 적용되는 데이터들은 key값을 기준으로 grouped된 데이터 기준

In [32]:
score = lambda x: (x.max())
grouped.transform(score) # 각 Team별로 모든 column이 max값 적용됨

,Rank,Year,Points
0,2,2017,876
1,2,2017,876
2,3,2015,863
3,3,2015,863
4,3,2017,788
5,4,2015,812
6,3,2017,788
7,3,2017,788
8,2,2017,876
9,4,2015,804


- normalization

In [34]:
score = lambda x: (x - x.mean()) / x.std()
grouped.transform(score).fillna(0)

,Rank,Year,Points
0,-1.500000,-1.161895,1.284327
1,0.500000,-0.387298,0.302029
2,-0.707107,-0.707107,0.707107
3,0.707107,0.707107,-0.707107
4,1.154701,-1.091089,-0.860862
5,0.000000,0.000000,0.000000
6,-0.577350,0.218218,-0.236043
7,-0.577350,0.872872,1.096905
8,0.500000,0.387298,-0.770596
9,0.707107,-0.707107,-0.707107


### 1-2-3) filter
- 특정 조건으로 데이터를 검색할때 사용  
- filter안에는 boolean 조건이 존재해야함  
- len(x)는 grouped된 dataframe 개수

In [35]:
df.groupby('Team').filter(lambda x: len(x) >= 3)

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
4,Kings,3,2014,741
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
11,Riders,2,2017,690


In [43]:
df.groupby('Team').filter(lambda x: x['Rank'].sum() > 5)

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
8,Riders,2,2016,694
11,Riders,2,2017,690


In [41]:
df.groupby('Team').filter(lambda x: x['Points'].mean() > 800)

,Team,Rank,Year,Points
5,kings,4,2015,812


In [44]:
df.groupby('Team').filter(lambda x: x['Rank'].mean() > 2)

,Team,Rank,Year,Points
2,Devils,2,2014,863
3,Devils,3,2015,673
5,kings,4,2015,812
9,Royals,4,2014,701
10,Royals,1,2015,804


## Case Study

In [45]:
import dateutil

In [48]:
df_phone = pd.read_csv('./resource/phone_data.csv') 
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [49]:
df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst=True)

df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


### groupby

In [52]:
df_phone.groupby('month')['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [55]:
df_phone.groupby(['item', 'month'])['duration'].sum()

item  month  
call  2014-11    25547.000
      2014-12    13561.000
      2015-01    17070.000
      2015-02    14416.000
      2015-03    21727.000
data  2014-11      998.441
      2014-12     1032.870
      2015-01     1067.299
      2015-02     1067.299
      2015-03      998.441
sms   2014-11       94.000
      2014-12       48.000
      2015-01       86.000
      2015-02       39.000
      2015-03       25.000
Name: duration, dtype: float64

In [53]:
df_phone[df_phone['item'] == 'call'].groupby('network')['duration'].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

In [54]:
df_phone.groupby(['month', 'item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [56]:
# 위의 결과를 matrix로
df_phone.groupby(['month', 'item'])['date'].count().unstack()

item,call,data,sms
month,,,
2014-11,107,29,94
2014-12,79,30,48
2015-01,88,31,86
2015-02,67,31,39
2015-03,47,29,25


### aggregation

In [58]:
df_phone.groupby('month', as_index=False).agg({'duration':'sum'})

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


In [60]:
df_phone.groupby('month').agg({'duration':'mean'})

,duration
month,
2014-11,115.823657
2014-12,93.260318
2015-01,88.894141
2015-02,113.301453
2015-03,225.251891


#### multiple aggregation operations

In [61]:
df_phone.groupby(['month', 'item']).agg({'duration':'sum', 'network_type': 'count', 'date':'first'})

duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

In [62]:
df_phone.groupby(['month', 'item']).agg({
    'duration' : [min, max, sum],
    'network_type' : 'count',
    'date' : [min, 'first', 'nunique']
})

duration                       network_type                date  \
                  min        max        sum        count                 min   
month   item                                                                   
2014-11 call    1.000   1940.000  25547.000          107 2014-10-15 06:58:00   
        data   34.429     34.429    998.441           29 2014-10-15 06:58:00   
        sms     1.000      1.000     94.000           94 2014-10-16 22:18:00   
2014-12 call    2.000   2120.000  13561.000           79 2014-11-14 17:24:00   
        data   34.429     34.429   1032.870           30 2014-11-13 06:58:00   
        sms     1.000      1.000     48.000           48 2014-11-14 17:28:00   
2015-01 call    2.000   1859.000  17070.000           88 2014-12-15 20:03:00   
        data   34.429     34.429   1067.299           31 2014-12-13 06:58:00   
        sms     1.000      1.000     86.000           86 2014-12-15 19:56:00   
2015-02 call    1.000   1863.000  14416.000           67 2015-01-15 10:36:00   
        data   34.429     34.429   1067.299           31 2015-01-13 06:58:00   
        sms     1.000      1.000     39.000           39 2015-01-15 12:23:00   
2015-03 call    2.000  10528.000  21727.000           47 2015-02-12 20:15:00   
        data   34.429     34.429    998.441           29 2015-02-13 06:58:00   
        sms     1.000      1.000     25.000           25 2015-02-19 18:46:00   

                                          
                           first nunique  
month   item                              
2014-11 call 2014-10-15 06:58:00     104  
        data 2014-10-15 06:58:00      29  
        sms  2014-10-16 22:18:00      79  
2014-12 call 2014-11-14 17:24:00      76  
        data 2014-11-13 06:58:00      30  
        sms  2014-11-14 17:28:00      41  
2015-01 call 2014-12-15 20:03:00      84  
        data 2014-12-13 06:58:00      31  
        sms  2014-12-15 19:56:00      58  
2015-02 call 2015-01-15 10:36:00      67  
        data 2015-01-13 06:58:00      31  
        sms  2015-01-15 12:23:00      27  
2015-03 call 2015-02-12 20:15:00      47  
        data 2015-02-13 06:58:00      29  
        sms  2015-02-19 18:46:00      17

### grouped

In [64]:
grouped = df_phone.groupby('month').agg({'duration' : [min, max, np.mean]})
grouped

duration                     
             min      max        mean
month                                
2014-11      1.0   1940.0  115.823657
2014-12      1.0   2120.0   93.260318
2015-01      1.0   1859.0   88.894141
2015-02      1.0   1863.0  113.301453
2015-03      1.0  10528.0  225.251891

In [63]:
grouped.columns = grouped.columns.droplevel(level=0) #0번째 level(여기선 duration) drop
grouped.rename(columns = {'min': 'min_duration', 'max' : 'max_duration', 'mean' : 'mean_duration'})

,min_duration,max_duration,mean_duration
month,,,
2014-11,1.0,1940.0,115.823657
2014-12,1.0,2120.0,93.260318
2015-01,1.0,1859.0,88.894141
2015-02,1.0,1863.0,113.301453
2015-03,1.0,10528.0,225.251891


### grouped + prefix

In [65]:
grouped = df_phone.groupby('month').agg({'duration' : [min, max, np.mean]})
grouped

duration                     
             min      max        mean
month                                
2014-11      1.0   1940.0  115.823657
2014-12      1.0   2120.0   93.260318
2015-01      1.0   1859.0   88.894141
2015-02      1.0   1863.0  113.301453
2015-03      1.0  10528.0  225.251891

In [66]:
grouped.columns = grouped.columns.droplevel(level=0)
grouped

,min,max,mean
month,,,
2014-11,1.0,1940.0,115.823657
2014-12,1.0,2120.0,93.260318
2015-01,1.0,1859.0,88.894141
2015-02,1.0,1863.0,113.301453
2015-03,1.0,10528.0,225.251891


In [67]:
grouped.add_prefix('duration_')

,duration_min,duration_max,duration_mean
month,,,
2014-11,1.0,1940.0,115.823657
2014-12,1.0,2120.0,93.260318
2015-01,1.0,1859.0,88.894141
2015-02,1.0,1863.0,113.301453
2015-03,1.0,10528.0,225.251891


# 2. Pivot Table & Crosstab

## 2-1) Pivot Table
- Excel의 pivot table과 동일  
- Index 축은 groupby와 동일  
- column에 추가로 labeling 값을 추가하여 value에 numeric type 값을 aggregation 하는 형태

In [68]:
df_phone = pd.read_csv('./resource/phone_data.csv')
df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst=True)
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [69]:
df_phone.pivot_table(['duration'],
                    index=[df_phone.month, df_phone.item],
                    columns=df_phone.network, aggfunc='sum', fill_value=0)

duration                                                   \
network        Meteor Tesco  Three Vodafone      data landline special   
month   item                                                             
2014-11 call     1521  4045  12458     4316     0.000     2906       0   
        data        0     0      0        0   998.441        0       0   
        sms        10     3     25       55     0.000        0       1   
2014-12 call     2010  1819   6316     1302     0.000     1424       0   
        data        0     0      0        0  1032.870        0       0   
        sms        12     1     13       18     0.000        0       0   
2015-01 call     2207  2904   6445     3626     0.000     1603       0   
        data        0     0      0        0  1067.299        0       0   
        sms        10     3     33       40     0.000        0       0   
2015-02 call     1188  4087   6279     1864     0.000      730       0   
        data        0     0      0        0  1067.299        0       0   
        sms         1     2     11       23     0.000        0       2   
2015-03 call      274   973   4966     3513     0.000    11770       0   
        data        0     0      0        0   998.441        0       0   
        sms         0     4      5       13     0.000        0       0   

                              
network      voicemail world  
month   item                  
2014-11 call       301     0  
        data         0     0  
        sms          0     0  
2014-12 call       690     0  
        data         0     0  
        sms          0     4  
2015-01 call       285     0  
        data         0     0  
        sms          0     0  
2015-02 call       268     0  
        data         0     0  
        sms          0     0  
2015-03 call       231     0  
        data         0     0  
        sms          0     3

## 2-2) Crosstab
- 특히 두 column의 교차 빈도, 비율, 덧셈 등을 구할 때 사용  
- Pivot table의 특수한 형태  
- User-Item Rating Matrix 등을 만들때 사용가능함

In [75]:
df_movie = pd.read_csv('./resource/movie_rating.csv')
df_movie.head(20)

,critic,title,rating
0,Jack Matthews,Lady in the Water,3.0
1,Jack Matthews,Snakes on a Plane,4.0
2,Jack Matthews,You Me and Dupree,3.5
3,Jack Matthews,Superman Returns,5.0
4,Jack Matthews,The Night Listener,3.0
5,Mick LaSalle,Lady in the Water,3.0
6,Mick LaSalle,Snakes on a Plane,4.0
7,Mick LaSalle,Just My Luck,2.0
8,Mick LaSalle,Superman Returns,3.0
9,Mick LaSalle,You Me and Dupree,2.0


In [72]:
pd.crosstab(index=df_movie.critic, columns=df_movie.title, values=df_movie.rating, aggfunc='first').fillna(0)

title,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,You Me and Dupree
critic,,,,,,
Claudia Puig,3.0,0.0,3.5,4.0,4.5,2.5
Gene Seymour,1.5,3.0,3.5,5.0,3.0,3.5
Jack Matthews,0.0,3.0,4.0,5.0,3.0,3.5
Lisa Rose,3.0,2.5,3.5,3.5,3.0,2.5
Mick LaSalle,2.0,3.0,4.0,3.0,3.0,2.0
Toby,0.0,0.0,4.5,4.0,0.0,1.0


- Using pivot table

In [73]:
df_movie.pivot_table(['rating'],
                    index=df_movie.critic,
                    columns=df_movie.title,
                    aggfunc='sum', fill_value=0)

rating                                      \
title         Just My Luck Lady in the Water Snakes on a Plane   
critic                                                           
Claudia Puig           3.0               0.0               3.5   
Gene Seymour           1.5               3.0               3.5   
Jack Matthews          0.0               3.0               4.0   
Lisa Rose              3.0               2.5               3.5   
Mick LaSalle           2.0               3.0               4.0   
Toby                   0.0               0.0               4.5   

                                                                     
title         Superman Returns The Night Listener You Me and Dupree  
critic                                                               
Claudia Puig               4.0                4.5               2.5  
Gene Seymour               5.0                3.0               3.5  
Jack Matthews              5.0                3.0               3.5  
Lisa Rose                  3.5                3.0               2.5  
Mick LaSalle               3.0                3.0               2.0  
Toby                       4.0                0.0               1.0

- Using Groupby

In [76]:
df_movie.groupby(['critic', 'title']).agg({'rating':'sum'}).unstack().fillna(0)

rating                                      \
title         Just My Luck Lady in the Water Snakes on a Plane   
critic                                                           
Claudia Puig           3.0               0.0               3.5   
Gene Seymour           1.5               3.0               3.5   
Jack Matthews          0.0               3.0               4.0   
Lisa Rose              3.0               2.5               3.5   
Mick LaSalle           2.0               3.0               4.0   
Toby                   0.0               0.0               4.5   

                                                                     
title         Superman Returns The Night Listener You Me and Dupree  
critic                                                               
Claudia Puig               4.0                4.5               2.5  
Gene Seymour               5.0                3.0               3.5  
Jack Matthews              5.0                3.0               3.5  
Lisa Rose                  3.5                3.0               2.5  
Mick LaSalle               3.0                3.0               2.0  
Toby                       4.0                0.0               1.0

# 3. Merge & Concatenate

## 3-1) Merge
- SQL에서 많이 사용하는 Merge와 같은 기능  
- Column을 기준으로 두 개의 데이터를 하나로 합침

In [77]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_score': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'test_score'])
df_a

,subject_id,test_score
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,7,14
6,8,15
7,9,1
8,10,61
9,11,16


In [78]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [79]:
pd.merge(df_a, df_b, on='subject_id')

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice
3,8,15,Betty,Btisan


In [80]:
pd.merge(df_a, df_b, left_on='subject_id', right_on='subject_id')

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice
3,8,15,Betty,Btisan


In [86]:
pd.merge(df_a, df_b, on='subject_id', how='left')

,subject_id,test_score,first_name,last_name
0,1,51,NaN,NaN
1,2,15,NaN,NaN
2,3,15,NaN,NaN
3,4,61,Billy,Bonder
4,5,16,Brian,Black
5,7,14,Bryce,Brice
6,8,15,Betty,Btisan
7,9,1,NaN,NaN
8,10,61,NaN,NaN
9,11,16,NaN,NaN


In [87]:
pd.merge(df_a, df_b, on='subject_id', how='right')

,subject_id,test_score,first_name,last_name
0,4,61.0,Billy,Bonder
1,5,16.0,Brian,Black
2,7,14.0,Bryce,Brice
3,8,15.0,Betty,Btisan
4,6,NaN,Bran,Balwner


In [88]:
pd.merge(df_a, df_b, on='subject_id', how='outer')

,subject_id,test_score,first_name,last_name
0,1,51.0,NaN,NaN
1,2,15.0,NaN,NaN
2,3,15.0,NaN,NaN
3,4,61.0,Billy,Bonder
4,5,16.0,Brian,Black
5,7,14.0,Bryce,Brice
6,8,15.0,Betty,Btisan
7,9,1.0,NaN,NaN
8,10,61.0,NaN,NaN
9,11,16.0,NaN,NaN


In [89]:
pd.merge(df_a, df_b, on='subject_id', how='inner')

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice
3,8,15,Betty,Btisan


### index based join

In [90]:
pd.merge(df_a, df_b, right_index=True, left_index=True)

,subject_id_x,test_score,subject_id_y,first_name,last_name
0,1,51,4,Billy,Bonder
1,2,15,5,Brian,Black
2,3,15,6,Bran,Balwner
3,4,61,7,Bryce,Brice
4,5,16,8,Betty,Btisan


## 3-2) Concatenate
- 같은 형태의 데이터를 붙이는 연산작업

In [95]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5'],
        'first_name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'], 
        'last_name': ['Anderson', 'Ackerman', 'Ali', 'Aoni', 'Atiches']}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_a

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches


In [96]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [97]:
df_new = pd.concat([df_a, df_b])
df_new.reset_index()

,index,subject_id,first_name,last_name
0,0,1,Alex,Anderson
1,1,2,Amy,Ackerman
2,2,3,Allen,Ali
3,3,4,Alice,Aoni
4,4,5,Ayoung,Atiches
5,0,4,Billy,Bonder
6,1,5,Brian,Black
7,2,6,Bran,Balwner
8,3,7,Bryce,Brice
9,4,8,Betty,Btisan


In [98]:
df_a.append(df_b)

,subject_id,first_name,last_name
0,1,Alex,Anderson
1,2,Amy,Ackerman
2,3,Allen,Ali
3,4,Alice,Aoni
4,5,Ayoung,Atiches
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [99]:
df_new = pd.concat([df_a, df_b], axis=1)
df_new.reset_index()

#이렇게 할바엔 merge를 많이 씀

,index,subject_id,first_name,last_name,subject_id,first_name,last_name
0,0,1,Alex,Anderson,4,Billy,Bonder
1,1,2,Amy,Ackerman,5,Brian,Black
2,2,3,Allen,Ali,6,Bran,Balwner
3,3,4,Alice,Aoni,7,Bryce,Brice
4,4,5,Ayoung,Atiches,8,Betty,Btisan


## Case Study

In [100]:
import os

In [102]:
files = [file_name for file_name in os.listdir('./resource') if file_name.endswith('xlsx')]
files.remove('excel-comp-data.xlsx')
files

['customer-status.xlsx',
 'sales-feb-2014.xlsx',
 'sales-jan-2014.xlsx',
 'sales-mar-2014.xlsx']

In [105]:
df_list = [pd.read_excel('./resource/' + df_filename) for df_filename in files]
status = df_list[0]
sales = pd.concat(df_list[1:])

In [106]:
status.head()

,account number,name,status
0,740150,Barton LLC,gold
1,714466,Trantow-Barrows,silver
2,218895,Kulas Inc,bronze
3,307599,"Kassulke, Ondricka and Metz",bronze
4,412290,Jerde-Hilpert,bronze


In [107]:
sales.head()

,account number,name,sku,quantity,unit price,ext price,date
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20


In [108]:
merge_df = pd.merge(status, sales, how='inner', on='account number')
merge_df.head()

,account number,name_x,status,name_y,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,gold,Barton LLC,S1-93683,21,10.34,217.14,2014-02-07 08:34:50
1,740150,Barton LLC,gold,Barton LLC,S2-10342,47,96.68,4543.96,2014-02-12 03:36:52
2,740150,Barton LLC,gold,Barton LLC,B1-38851,3,79.49,238.47,2014-02-12 05:09:16
3,740150,Barton LLC,gold,Barton LLC,B1-20000,28,81.39,2278.92,2014-02-15 07:45:16
4,740150,Barton LLC,gold,Barton LLC,B1-38851,17,81.22,1380.74,2014-02-17 17:12:16


In [110]:
merge_df.groupby(['status', 'name_x'])['quantity', 'ext price'].sum()

quantity  ext price
status name_x                                              
bronze Jerde-Hilpert                         581   30617.51
       Kassulke, Ondricka and Metz           366   20543.71
       Kulas Inc                             738   53507.36
       Pollich LLC                           219    9298.02
       Sanford and Sons                      403   24003.93
gold   Barton LLC                            346   21909.13
       Cronin, Oberbrunner and Spencer       543   26809.63
       Herman LLC                            417   22940.82
       Stokes LLC                            449   25680.43
silver Frami, Hills and Schmidt              338   19634.31
       Keeling LLC                           438   27341.41
       Kiehn-Spinka                          350   21027.06
       Koepp Ltd                             315   17381.89
       Kuhn-Gusikowski                       558   29994.43
       Trantow-Barrows                       643   35354.61
       White-Trantow                         556   34070.80

In [114]:
merge_df.groupby(['status', 'name_x'])['quantity', 'ext price'].sum().reset_index().sort_values(by=['status', 'name_x', 'quantity'], ascending=False)

#그냥 sort_values만 하면 앞의 status groupby가 깨지기 때문에 groupby다음 reset_index() 해줘야함

,status,name_x,quantity,ext price
15,silver,White-Trantow,556,34070.80
14,silver,Trantow-Barrows,643,35354.61
13,silver,Kuhn-Gusikowski,558,29994.43
12,silver,Koepp Ltd,315,17381.89
11,silver,Kiehn-Spinka,350,21027.06
10,silver,Keeling LLC,438,27341.41
9,silver,"Frami, Hills and Schmidt",338,19634.31
8,gold,Stokes LLC,449,25680.43
7,gold,Herman LLC,417,22940.82
6,gold,"Cronin, Oberbrunner and Spencer",543,26809.63


# 4. Database Connection & Persistance

## 4-1) Database Connection
- Data loading시 db connection 기능을 제공함  

### Example from 
- https://www.dataquest.io/blog/python-pandas-databases/

In [2]:
conn = lite.connect('./resource/flights.db')
cur = conn.cursor()
cur.execute("SELECT * FROM airlines limit 5;") # airlines에서 데이터를 5개까지 select
results = cur.fetchall()
results

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'),
 (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'),
 (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'),
 (3,
  '4',
  '2 Sqn No 1 Elementary Flying Training School',
  '\\N',
  None,
  'WYT',
  None,
  'United Kingdom',
  'N'),
 (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N')]

In [117]:
query = """
SELECT name, sql
FROM sqlite_master;
"""

pd.read_sql(query, conn)

,name,sql
0,airports,"CREATE TABLE airports (\n[index] INTEGER,\n [..."
1,ix_airports_index,CREATE INDEX ix_airports_index ON airports ([i...
2,airlines,"CREATE TABLE airlines (\n[index] INTEGER,\n [..."
3,ix_airlines_index,CREATE INDEX ix_airlines_index ON airlines ([i...
4,routes,"CREATE TABLE routes (\n[index] INTEGER,\n [ai..."
5,ix_routes_index,CREATE INDEX ix_routes_index ON routes ([index])


In [3]:
df_airlines = pd.read_sql_query("SELECT * FROM airlines;", conn)
df_airlines.head()

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [4]:
df_airports = pd.read_sql_query("SELECT * FROM airports;", conn)
df_airports.head()

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


In [5]:
df_routes = pd.read_sql_query("SELECT * FROM routes;", conn)
df_routes.head()

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


## 4-2) Pandas Persistence
- Xls 엔진으로 openpyxls 또는 XlsxWriter 사용
- See more : http://xlsxwriter.readthedocs.io/working_with_pandas.html

In [8]:
writer = pd.ExcelWriter('./resource/df_routes.xlsx', engine='xlsxwriter')
df_routes.to_excel(writer, sheet_name='Sheet1')

## 4-3) Pickle Persistence
- 가장 일반적인 python 파일 persistence  
- to_pickle, read_pickle 함수 사용

In [9]:
df_routes.to_pickle('./resource/df_routes.pickle')

In [10]:
df_routes_pickle = pd.read_pickle('./resource/df_routes.pickle')
df_routes_pickle.head()

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2
2,2,2B,410,ASF,2966,MRV,2962,None,0,CR2
3,3,2B,410,CEK,2968,KZN,2990,None,0,CR2
4,4,2B,410,CEK,2968,OVB,4078,None,0,CR2


In [11]:
df_routes_pickle.describe()

,index
count,67663.000000
mean,33831.000000
std,19532.769969
min,0.000000
25%,16915.500000
50%,33831.000000
75%,50746.500000
max,67662.000000
